## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

====================================================================================================

## 1

import libs \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file\
memory allocations to manage ghosts


====================================================================================================

In [2]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import csv


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T()

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc()

agent_cache={} # dict with uid as keys and agents' tuples as values

===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [3]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, capital:float, labor:float,\
                 duration:int, recipe: float, laborProductivity: float, dimension: int, addedValue: float,\
                 initialInventories: float, finalInventories: float):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        
        self.capital=capital
        self.labor=labor
        self.duration=duration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.dimension=dimension
        
        self.output=0
        self.productionClock=1
        self.multiperiodalCosts=0
        #print("AA", self.uid,self.labor, self.capital, self.duration, self.recipe, self.dimension)
        #print(self.uid, self.dimension)
        
        self.addedValue=addedValue
        self.initalInventories=initialInventories
        self.finalInventories=finalInventories
        
    def produce(self, production:float)->tuple:

        unusedLabor=self.labor
        unusedCapital=self.capital
        
        self.targetProductionOfThePeriod=production/self.duration
        requiredLabor=self.targetProductionOfThePeriod/self.laborProductivity
        requiredCapital=requiredLabor*(1/self.recipe)
        
        if requiredLabor <= self.labor and requiredCapital <= self.capital: 

            if self.productionClock != self.duration:
                costOfProduction= params['wage']*requiredLabor+params['costOfCapital']*requiredCapital
                costOfUnusedFactors=  params['wage']*unusedLabor+params['costOfCapital']*unusedCapital 
                self.multiperiodalCosts += costOfProduction
                self.productionClock += 1
            else: 
                self.productionClock = 1
                self.multiperiodalCosts=0
 
            self.output=self.targetProductionOfThePeriod
            unusedLabor=self.labor-requiredLabor
            unusedCapital=self.capital-requiredCapital
            
        else:
            self.output=0
            requiredLabor=0
            requiredCapital=0
            

        
        self.lostProduction=0
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.lostProduction=self.multiperiodalCosts
            self.productionClock = 0
            
        #print("firm production", self.uid, self.dimension, production, self.output, self.duration, self.labor, self.capital, \
              #unusedLabor, unusedCapital, self.recipe, flush=True)
        
        """
        if self.productionClock!=self.duration:
            #facciamo scorte
        else:
            #vendiamo
        """
          
            
        return (self.output, costOfProduction, costOfUnusedFactors, self.lostProduction)
    
            

===================================================================================================

## 3

the model

===================================================================================================

In [4]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalLostProduction=[]
        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initGhosts) 
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L-min, L-max, K-min, K-max, t-min, t-max, recipe, L-prod
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=0
            k=0
            for row in firmReader:
                print(row)
                for i in range(int(row[0] * params['Firm.count'])// rankNum):
                    labor= rng.integers(row[1], row[2]+1)
                    capital= row[3] + rng.random()*(row[4] -row[3])
                    duration= rng.integers(row[5], row[6]+1)
                    recipe= row[7] #L/K 
                    laborProductivity= row[8]
                    dimension=self.rowNumber
                    addedValue=0
                    initialInventories=0
                    finalInventories=0
                    aFirm =Firm(k, rank, capital, labor, duration, recipe, laborProductivity, dimension, addedValue,\
                               initialInventories, finalInventories)
                    context.add(aFirm)
                    k += 1
                self.rowNumber += 1

        
    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber))
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            #print(aFirm.dimension)
            tupleOfProductionResults = aFirm.produce(rng.random()*20)
            self.totalProduction[t()][aFirm.dimension] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.dimension] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.dimension] += tupleOfProductionResults[2]
            self.totalLostProduction[t()][aFirm.dimension] += tupleOfProductionResults[3]
          
            
    #finish
    def finish(self):
        
        # infos for data_analysis.ipynb
        with open('plotInfo.csv', 'w', newline='')\
          as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum))
        
        
        print("\n total production", self.totalProduction, flush=True)
        print("\n cost of production", self.totalCostOfProduction, flush=True)
        print("\n total cost of unused factors", self.totalCostOfUnusedFactors, flush=True)
        print("\n total lost production", self.totalLostProduction, flush=True)

        print("THE END!", flush=True)
        
        names=["_production_","_cost of production_","_total cost of unused factors_",
              "_total lost production_"]
        contents=[self.totalProduction,self.totalCostOfProduction,\
                  self.totalCostOfUnusedFactors,self.totalLostProduction]
        
        for s in range(4):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w',\
                  newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

==========================================================================================================

In [5]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

[0.1, 250.0, 600.0, 80.0, 100.0, 5.0, 10.0, 5.0, 0.8]
[0.3, 50.0, 300.0, 40.0, 60.0, 2.0, 4.0, 2.0, 0.7]
[0.6, 5.0, 40.0, 20.0, 40.0, 1.0, 1.0, 0.5, 0.6]
rank 0 tick 0
rank 0 tick 1
rank 0 tick 2
rank 0 tick 3
rank 0 tick 4
rank 0 tick 5
rank 0 tick 6
rank 0 tick 7
rank 0 tick 8
rank 0 tick 9
rank 0 tick 10

 total production [[12.990597423899027, 105.57039401885169, 61.13632401792782], [13.14860777372323, 113.01750187604719, 78.89386927352999], [12.243533013580013, 110.95549571315128, 126.42533575917433], [8.461147282800376, 99.75688432743125, 81.54463956003009], [10.985423833128413, 95.1337799510421, 94.10571372621484], [15.967323854048455, 127.2000749195736, 86.88957637332425], [13.875258366546806, 115.80871625962116, 91.3420206883764], [17.740463525227508, 103.18271656383462, 88.79388477625241], [14.477212137755451, 105.21909847688254, 109.36439239099005], [12.27256121559365, 101.40716954019753, 115.30870167033268]]

 cost of production [[16.56301171547126, 158.3555910282776, 122.2